In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, SimpleRNN, Dropout, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.constraints import maxnorm

# Fake detection

##  Passive Agressive Classifier

### Data preparation

In [2]:
df = pd.read_csv("news.csv")
df.head(10)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [3]:
conversion_dict = {'REAL': 0, 'FAKE': 1}
df['label'] = df['label'].replace(conversion_dict)
df.label.value_counts()

0    3171
1    3164
Name: label, dtype: int64

In [4]:
df.head(10)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",1
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",1
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,0
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,0
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,0


### Words vectorize and model build

In [5]:
x_train_first,x_test_first,y_train_first,y_test_first=train_test_split(df['text'], df['label'], test_size=0.25, random_state=7, shuffle=True)
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.75 )

In [6]:
first_vec_train=tfidf_vectorizer.fit_transform(x_train_first.values.astype('U'))
first_vec_test=tfidf_vectorizer.transform(x_test_first.values.astype('U'))
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(first_vec_train,y_train_first)

PassiveAggressiveClassifier(max_iter=50)

### Results

In [7]:
y_pred=pac.predict(first_vec_test)
score=accuracy_score(y_test_first,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 92.49%


In [8]:
confusion_matrix(y_test_first,y_pred)

array([[717,  59],
       [ 60, 748]])

In [9]:
X=tfidf_vectorizer.transform(df['text'].values.astype('U'))
scores = cross_val_score(pac, X, df['label'].values, cv=5)
print(f'Accuracy: {round(scores.mean()*100,2)}%')

Accuracy: 93.83%


### Try another way to represent words

In [10]:
vectorizer = CountVectorizer()
count_vector_first = vectorizer.fit_transform(x_train_first.values.astype('U'))
first_count_test=vectorizer.transform(x_test_first.values.astype('U'))
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(count_vector_first,y_train_first)

PassiveAggressiveClassifier(max_iter=50)

In [11]:
y_pred=pac.predict(first_count_test)
score=accuracy_score(y_test_first,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 90.78%


In [12]:
confusion_matrix(y_test_first,y_pred)

array([[727,  49],
       [ 97, 711]])

In [13]:
X=vectorizer.transform(df['text'].values.astype('U'))
scores = cross_val_score(pac, X, df['label'].values, cv=5)
print(f'Accuracy: {round(scores.mean()*100,2)}%')

/home/lurakil/.local/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


Accuracy: 90.58%


### Trying another dataset

In [14]:
df_true=pd.read_csv('True.csv', nrows=2500)
df_true['label']= 0
df_true_rep=[df_true['text'][i].replace('WASHINGTON (Reuters) - ','').replace('LONDON (Reuters) - ','').replace('(Reuters) - ','') for i in range(len(df_true['text']))]
df_true['text']=df_true_rep
df_fake = pd.read_csv("Fake.csv" ,nrows=2500)
df_fake['label']= 1
df_final=pd.concat([df_true,df_fake])
df_final=df_final.drop(['subject','date'], axis=1)

In [15]:
df_final.head(10)

,title,text,label
0,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction ...,0
1,U.S. military to accept transgender recruits o...,Transgender people will be allowed for the fir...,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,The special counsel investigation of links bet...,0
3,FBI Russia probe helped by Australian diplomat...,Trump campaign adviser George Papadopoulos tol...,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/President Donald Trump called on the U...,0
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./The White House said on ...",0
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla President Donald Trump sa...",0
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,0
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,0
9,Alabama official to certify Senator-elect Jone...,Alabama Secretary of State John Merrill said h...,0


In [16]:
df_final.tail(10)

,title,text,label
2490,REPORT: Even Republicans Are ‘Disturbed’ By T...,Donald Trump s days in the White House could b...,1
2491,Eric Trump Just Asked The Labor Department To...,"If any other American company did this, Donald...",1
2492,REPORT: Trump Considering Using National Guar...,Donald Trump reportedly wants to use the Natio...,1
2493,WATCH: Trump Assumes Black Reporter Can Set U...,After claiming that he s the least racist per...,1
2494,Trump Thinks Rush Limbaugh Is Real News And T...,Rush Limbaugh praised Donald Trump s insanely ...,1
2495,"WATCH: Fox News Host DEFENDS CNN Reporter, Te...",Even Fox News is calling Donald Trump out for ...,1
2496,GOP Senator DESPERATELY Worried About Trump’s...,"On Thursday afternoon, the world was treated t...",1
2497,WATCH: NBC Reporter Calls Trump Out For Lying...,Donald Trump finally held his first solo press...,1
2498,Tapper SLAMS Trump For BIZARRE Behavior At ‘W...,There is certainly no love lost between CNN an...,1
2499,LOL: Putin Is Angry Now Because Trump Gets Mo...,Vladimir Putin is super pissed because Russian...,1


### Model build and words vecrorize

In [17]:
x_train_second,x_test_second,y_train_second,y_test_second=train_test_split(df_final['text'], df_final['label'], test_size=0.25)
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.75 )

In [18]:
second_vec_train=tfidf_vectorizer.fit_transform(x_train_second.values.astype('U')) 
second_vec_test=tfidf_vectorizer.transform(x_test_second.values.astype('U'))
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(second_vec_train, y_train_second)

PassiveAggressiveClassifier(max_iter=50)

### Results

In [19]:
y_pred=pac.predict(second_vec_test)
score=accuracy_score(y_test_second,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 99.28%


In [20]:
confusion_matrix(y_test_second,y_pred)

array([[609,   4],
       [  5, 632]])

In [21]:
X=tfidf_vectorizer.transform(df['text'].values.astype('U'))
scores = cross_val_score(pac, X, df['label'].values, cv=5)
print(f'Accuracy: {round(scores.mean()*100,2)}%')

Accuracy: 93.58%


### Try another way to represent words

In [22]:
vectorizer = CountVectorizer()
count_vector_second = vectorizer.fit_transform(x_train_second.values.astype('U'))
second_count_test=vectorizer.transform(x_test_second.values.astype('U'))
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(count_vector_second,y_train_second)

PassiveAggressiveClassifier(max_iter=50)

In [23]:
y_pred=pac.predict(second_count_test)
score=accuracy_score(y_test_second,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 98.48%


In [24]:
confusion_matrix(y_test_second,y_pred)

array([[608,   5],
       [ 14, 623]])

In [25]:
X=vectorizer.transform(df_final['text'].values.astype('U'))
scores = cross_val_score(pac, X, df_final['label'].values, cv=5)
print(f'Accuracy: {round(scores.mean()*100,2)}%')

Accuracy: 98.84%


## Random Forest Classfier

In [26]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(first_vec_train, y_train_first)

RandomForestClassifier(random_state=0)

In [27]:
y_pred=RFC.predict(first_vec_test)
score=accuracy_score(y_test_first,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 90.59%


In [28]:
confusion_matrix(y_test_first,y_pred)

array([[715,  61],
       [ 88, 720]])

In [29]:
X=tfidf_vectorizer.transform(df['text'].values.astype('U'))
scores = cross_val_score(RFC, X, df['label'].values, cv=5)
print(f'Accuracy: {round(scores.mean()*100,2)}%')

Accuracy: 90.47%


### Try another way to represent words

In [30]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(count_vector_first, y_train_first)

RandomForestClassifier(random_state=0)

In [31]:
y_pred=RFC.predict(first_count_test)
score=accuracy_score(y_test_first,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 87.88%


In [32]:
confusion_matrix(y_test_first,y_pred)

array([[674, 102],
       [ 90, 718]])

In [33]:
X=vectorizer.transform(df['text'].values.astype('U'))
scores = cross_val_score(RFC, X, df['label'].values, cv=5)
print(f'Accuracy: {round(scores.mean()*100,2)}%')

Accuracy: 88.79%


### Second dataset

In [34]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(second_vec_train, y_train_second)

RandomForestClassifier(random_state=0)

In [35]:
y_pred=RFC.predict(second_vec_test)
score=accuracy_score(y_test_second,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 99.68%


In [36]:
confusion_matrix(y_test_second,y_pred)

array([[610,   3],
       [  1, 636]])

In [37]:
X=tfidf_vectorizer.transform(df_final['text'].values.astype('U'))
scores = cross_val_score(RFC, X, df_final['label'].values, cv=5)
print(f'Accuracy: {round(scores.mean()*100,2)}%')

Accuracy: 99.28%


### Try another way to represent words

In [38]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(count_vector_second, y_train_second)

RandomForestClassifier(random_state=0)

In [39]:
y_pred=RFC.predict(second_count_test)
score=accuracy_score(y_test_second,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 99.6%


In [40]:
confusion_matrix(y_test_second,y_pred)

array([[611,   2],
       [  3, 634]])

In [41]:
X=vectorizer.transform(df_final['text'].values.astype('U'))
scores = cross_val_score(RFC, X, df_final['label'].values, cv=5)
print(f'Accuracy: {round(scores.mean()*100,2)}%')

Accuracy: 99.3%


# Neural Networks

## Data preparation

In [42]:
df_train = df.sample(frac=.8, random_state=11)
df_test = df.drop(df_train.index)

In [43]:
y_train = df_train['label'].values
y_test = df_test['label'].values

In [44]:
X_train = df_train['text'].values
X_test = df_test['text'].values

In [45]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_dict = tokenizer.index_word
print(len(word_dict))

87596


In [46]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [47]:
X_train_pad = pad_sequences(X_train_seq, maxlen=512, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=512, padding='post')

In [48]:
laenge_pads = 512
vocab_size = len(word_dict)
output_dim=64

## Simple LSTM

In [50]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size+1, output_dim=output_dim, input_length=laenge_pads))
lstm_model.add(Dropout(0.1))
lstm_model.add(LSTM(128))
lstm_model.add(Dropout(0.1))
lstm_model.add(Dropout(0.1))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 64)           5606208   
_________________________________________________________________
dropout (Dropout)            (None, 512, 64)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 5,705,153
Trainable params: 5,705,153
Non-trainable params: 0
____________________________________________

In [51]:
history = lstm_model.fit(X_train_pad, y_train, epochs=200, batch_size=64, 
                        validation_data=(X_test_pad, y_test))

Epoch 1/200
80/80 [==============================] - 48s 603ms/step - loss: 0.6370 - accuracy: 0.6667 - val_loss: 0.6071 - val_accuracy: 0.7443
Epoch 2/200
80/80 [==============================] - 47s 593ms/step - loss: 0.5110 - accuracy: 0.7747 - val_loss: 0.4709 - val_accuracy: 0.7845
Epoch 3/200
80/80 [==============================] - 48s 603ms/step - loss: 0.3780 - accuracy: 0.8285 - val_loss: 0.4367 - val_accuracy: 0.8122
Epoch 4/200
80/80 [==============================] - 49s 611ms/step - loss: 0.3712 - accuracy: 0.8512 - val_loss: 0.5432 - val_accuracy: 0.7348
Epoch 5/200
80/80 [==============================] - 50s 622ms/step - loss: 0.3885 - accuracy: 0.8236 - val_loss: 0.5507 - val_accuracy: 0.7096
Epoch 6/200
80/80 [==============================] - 49s 616ms/step - loss: 0.3556 - accuracy: 0.8408 - val_loss: 0.5127 - val_accuracy: 0.7593
Epoch 7/200
80/80 [==============================] - 48s 604ms/step - loss: 0.3027 - accuracy: 0.8613 - val_loss: 0.5150 - val_accuracy:

80/80 [==============================] - 53s 657ms/step - loss: 0.0498 - accuracy: 0.9893 - val_loss: 0.5517 - val_accuracy: 0.8674
Epoch 58/200
80/80 [==============================] - 53s 658ms/step - loss: 0.0517 - accuracy: 0.9893 - val_loss: 0.5529 - val_accuracy: 0.8706
Epoch 59/200
80/80 [==============================] - 53s 658ms/step - loss: 0.0477 - accuracy: 0.9895 - val_loss: 0.5818 - val_accuracy: 0.8690
Epoch 60/200
80/80 [==============================] - 53s 657ms/step - loss: 0.0571 - accuracy: 0.9876 - val_loss: 0.5770 - val_accuracy: 0.8556
Epoch 61/200
80/80 [==============================] - 53s 656ms/step - loss: 0.0754 - accuracy: 0.9826 - val_loss: 0.5472 - val_accuracy: 0.8343
Epoch 62/200
80/80 [==============================] - 52s 656ms/step - loss: 0.0559 - accuracy: 0.9890 - val_loss: 0.5737 - val_accuracy: 0.8619
Epoch 63/200
80/80 [==============================] - 52s 656ms/step - loss: 0.0632 - accuracy: 0.9858 - val_loss: 0.6331 - val_accuracy: 0.854

80/80 [==============================] - 53s 658ms/step - loss: 0.0225 - accuracy: 0.9963 - val_loss: 0.6343 - val_accuracy: 0.8650
Epoch 114/200
80/80 [==============================] - 53s 659ms/step - loss: 0.0219 - accuracy: 0.9963 - val_loss: 0.6634 - val_accuracy: 0.8635
Epoch 115/200
80/80 [==============================] - 53s 658ms/step - loss: 0.0228 - accuracy: 0.9963 - val_loss: 0.6489 - val_accuracy: 0.8627
Epoch 116/200
80/80 [==============================] - 53s 661ms/step - loss: 0.0230 - accuracy: 0.9963 - val_loss: 0.6794 - val_accuracy: 0.8635
Epoch 117/200
80/80 [==============================] - 53s 662ms/step - loss: 0.0227 - accuracy: 0.9963 - val_loss: 0.6369 - val_accuracy: 0.8658
Epoch 118/200
80/80 [==============================] - 53s 662ms/step - loss: 0.0234 - accuracy: 0.9961 - val_loss: 0.6543 - val_accuracy: 0.8658
Epoch 119/200
80/80 [==============================] - 53s 664ms/step - loss: 0.0226 - accuracy: 0.9963 - val_loss: 0.6722 - val_accuracy:

80/80 [==============================] - 54s 670ms/step - loss: 0.0287 - accuracy: 0.9947 - val_loss: 0.6693 - val_accuracy: 0.8587
Epoch 170/200
80/80 [==============================] - 54s 670ms/step - loss: 0.0217 - accuracy: 0.9963 - val_loss: 0.6312 - val_accuracy: 0.8706
Epoch 171/200
80/80 [==============================] - 54s 670ms/step - loss: 0.0198 - accuracy: 0.9966 - val_loss: 0.6564 - val_accuracy: 0.8650
Epoch 172/200
80/80 [==============================] - 54s 670ms/step - loss: 0.0188 - accuracy: 0.9968 - val_loss: 0.6646 - val_accuracy: 0.8682
Epoch 173/200
80/80 [==============================] - 54s 670ms/step - loss: 0.0181 - accuracy: 0.9968 - val_loss: 0.6726 - val_accuracy: 0.8698
Epoch 174/200
80/80 [==============================] - 54s 672ms/step - loss: 0.0187 - accuracy: 0.9968 - val_loss: 0.6800 - val_accuracy: 0.8690
Epoch 175/200
80/80 [==============================] - 54s 670ms/step - loss: 0.0190 - accuracy: 0.9968 - val_loss: 0.6870 - val_accuracy:

In [67]:
pred_y = lstm_model.predict_classes(X_test_pad)
score=accuracy_score(y_test,pred_y)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 88.0%


In [53]:
confusion_matrix(y_test,pred_y)

array([[523,  88],
       [ 64, 592]])

## Simple RNN

In [54]:
rnn_model = Sequential()
rnn_model.add(Embedding(input_dim = vocab_size + 1, output_dim = output_dim, input_length = laenge_pads))
rnn_model.add(SimpleRNN(output_dim, return_sequences=True, kernel_constraint=maxnorm(50)))
rnn_model.add(Dropout(0.1))
rnn_model.add(SimpleRNN(output_dim, return_sequences=True, kernel_constraint=maxnorm(50)))
rnn_model.add(Dropout(0.1))
rnn_model.add(SimpleRNN(output_dim))
rnn_model.add(Dropout(0.1))
rnn_model.add(Dense(1,activation='sigmoid'))
rnn_model.compile(optimizer='sgd', loss='mse',  metrics=['accuracy'])
rnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 512, 64)           5606208   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 512, 64)           8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 512, 64)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 512, 64)           8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 512, 64)           0         
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)               

In [55]:
history = rnn_model.fit(X_train_pad, y_train, epochs=200, batch_size=64, 
                        validation_data=(X_test_pad, y_test))

Epoch 1/200
80/80 [==============================] - 30s 370ms/step - loss: 0.2408 - accuracy: 0.6166 - val_loss: 0.2265 - val_accuracy: 0.6440
Epoch 2/200
80/80 [==============================] - 28s 354ms/step - loss: 0.2357 - accuracy: 0.6298 - val_loss: 0.2262 - val_accuracy: 0.6464
Epoch 3/200
80/80 [==============================] - 27s 342ms/step - loss: 0.2335 - accuracy: 0.6306 - val_loss: 0.2227 - val_accuracy: 0.6527
Epoch 4/200
80/80 [==============================] - 28s 349ms/step - loss: 0.2310 - accuracy: 0.6389 - val_loss: 0.2255 - val_accuracy: 0.6511
Epoch 5/200
80/80 [==============================] - 28s 355ms/step - loss: 0.2294 - accuracy: 0.6405 - val_loss: 0.2212 - val_accuracy: 0.6535
Epoch 6/200
80/80 [==============================] - 29s 357ms/step - loss: 0.2265 - accuracy: 0.6452 - val_loss: 0.2201 - val_accuracy: 0.6606
Epoch 7/200
80/80 [==============================] - 29s 358ms/step - loss: 0.2223 - accuracy: 0.6527 - val_loss: 0.2194 - val_accuracy:

80/80 [==============================] - 28s 347ms/step - loss: 0.2075 - accuracy: 0.6837 - val_loss: 0.2017 - val_accuracy: 0.7174
Epoch 58/200
80/80 [==============================] - 28s 347ms/step - loss: 0.2047 - accuracy: 0.6902 - val_loss: 0.1941 - val_accuracy: 0.7088
Epoch 59/200
80/80 [==============================] - 28s 347ms/step - loss: 0.2017 - accuracy: 0.6981 - val_loss: 0.1908 - val_accuracy: 0.7245
Epoch 60/200
80/80 [==============================] - 28s 347ms/step - loss: 0.2012 - accuracy: 0.6967 - val_loss: 0.2621 - val_accuracy: 0.6519
Epoch 61/200
80/80 [==============================] - 28s 346ms/step - loss: 0.1991 - accuracy: 0.7078 - val_loss: 0.2308 - val_accuracy: 0.6756
Epoch 62/200
80/80 [==============================] - 28s 347ms/step - loss: 0.1962 - accuracy: 0.7097 - val_loss: 0.1893 - val_accuracy: 0.7238
Epoch 63/200
80/80 [==============================] - 28s 347ms/step - loss: 0.1969 - accuracy: 0.7111 - val_loss: 0.2034 - val_accuracy: 0.685

80/80 [==============================] - 28s 353ms/step - loss: 0.2471 - accuracy: 0.5503 - val_loss: 0.2436 - val_accuracy: 0.5809
Epoch 114/200
80/80 [==============================] - 28s 353ms/step - loss: 0.2482 - accuracy: 0.5377 - val_loss: 0.2435 - val_accuracy: 0.5706
Epoch 115/200
80/80 [==============================] - 28s 351ms/step - loss: 0.2468 - accuracy: 0.5505 - val_loss: 0.2442 - val_accuracy: 0.5675
Epoch 116/200
80/80 [==============================] - 28s 351ms/step - loss: 0.2488 - accuracy: 0.5359 - val_loss: 0.2431 - val_accuracy: 0.5777
Epoch 117/200
80/80 [==============================] - 28s 351ms/step - loss: 0.2460 - accuracy: 0.5507 - val_loss: 0.2448 - val_accuracy: 0.5675
Epoch 118/200
80/80 [==============================] - 28s 351ms/step - loss: 0.2465 - accuracy: 0.5503 - val_loss: 0.2457 - val_accuracy: 0.5635
Epoch 119/200
80/80 [==============================] - 27s 333ms/step - loss: 0.2455 - accuracy: 0.5570 - val_loss: 0.2431 - val_accuracy:

80/80 [==============================] - 28s 349ms/step - loss: 0.2418 - accuracy: 0.5880 - val_loss: 0.2432 - val_accuracy: 0.5777
Epoch 170/200
80/80 [==============================] - 28s 350ms/step - loss: 0.2482 - accuracy: 0.5481 - val_loss: 0.2445 - val_accuracy: 0.5572
Epoch 171/200
80/80 [==============================] - 31s 388ms/step - loss: 0.2460 - accuracy: 0.5377 - val_loss: 0.2447 - val_accuracy: 0.5651
Epoch 172/200
80/80 [==============================] - 31s 390ms/step - loss: 0.2459 - accuracy: 0.5479 - val_loss: 0.2447 - val_accuracy: 0.5580
Epoch 173/200
80/80 [==============================] - 29s 363ms/step - loss: 0.2463 - accuracy: 0.5385 - val_loss: 0.2438 - val_accuracy: 0.5572
Epoch 174/200
80/80 [==============================] - 28s 349ms/step - loss: 0.2440 - accuracy: 0.5572 - val_loss: 0.2439 - val_accuracy: 0.5675
Epoch 175/200
80/80 [==============================] - 28s 348ms/step - loss: 0.2509 - accuracy: 0.5495 - val_loss: 0.2595 - val_accuracy:

In [66]:
pred_y = rnn_model.predict_classes(X_test_pad)
score=accuracy_score(y_test,pred_y)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 54.14%


In [57]:
confusion_matrix(y_test,pred_y)

array([[ 78, 533],
       [ 48, 608]])

## Recurrent Convolutional Neural Networks

In [60]:
conv_lstm_model = Sequential()
conv_lstm_model.add(Embedding(input_dim=vocab_size+1, output_dim=output_dim, input_length=laenge_pads))
conv_lstm_model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
conv_lstm_model.add(MaxPooling1D(pool_size=2))
conv_lstm_model.add(Conv1D(filters=512, kernel_size=5, padding='same', activation='relu'))
conv_lstm_model.add(MaxPooling1D(pool_size=2))
conv_lstm_model.add(Conv1D(filters=768, kernel_size=5, padding='same', activation='relu'))
conv_lstm_model.add(MaxPooling1D(pool_size=2))
conv_lstm_model.add(LSTM(128, return_sequences=True))
conv_lstm_model.add(Dropout(0.1))
conv_lstm_model.add(LSTM(128))
conv_lstm_model.add(Dropout(0.1))
conv_lstm_model.add(Dense(1, activation='sigmoid'))

conv_lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
conv_lstm_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 512, 64)           5606208   
_________________________________________________________________
conv1d (Conv1D)              (None, 512, 256)          82176     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 256, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 256, 512)          655872    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 128, 512)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 128, 768)          1966848   
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 64, 768)          

In [62]:
history = conv_lstm_model.fit(X_train_pad, y_train, epochs=100, batch_size=64, 
                        validation_data=(X_test_pad, y_test))

Epoch 1/100
80/80 [==============================] - 213s 3s/step - loss: 0.0652 - accuracy: 0.9797 - val_loss: 0.2728 - val_accuracy: 0.9227
Epoch 2/100
80/80 [==============================] - 218s 3s/step - loss: 0.0364 - accuracy: 0.9907 - val_loss: 0.3311 - val_accuracy: 0.9227
Epoch 3/100
80/80 [==============================] - 218s 3s/step - loss: 0.0220 - accuracy: 0.9953 - val_loss: 0.2811 - val_accuracy: 0.9305
Epoch 4/100
80/80 [==============================] - 220s 3s/step - loss: 0.0370 - accuracy: 0.9899 - val_loss: 0.4477 - val_accuracy: 0.8429
Epoch 5/100
80/80 [==============================] - 222s 3s/step - loss: 0.0167 - accuracy: 0.9970 - val_loss: 0.3914 - val_accuracy: 0.9124
Epoch 6/100
80/80 [==============================] - 223s 3s/step - loss: 0.1163 - accuracy: 0.9670 - val_loss: 0.4311 - val_accuracy: 0.8493
Epoch 7/100
80/80 [==============================] - 222s 3s/step - loss: 0.0868 - accuracy: 0.9773 - val_loss: 0.2982 - val_accuracy: 0.9187
Epoch 

Epoch 58/100
80/80 [==============================] - 271s 3s/step - loss: 1.2301e-05 - accuracy: 1.0000 - val_loss: 0.7990 - val_accuracy: 0.9084
Epoch 59/100
80/80 [==============================] - 231s 3s/step - loss: 1.1592e-05 - accuracy: 1.0000 - val_loss: 0.8039 - val_accuracy: 0.9084
Epoch 60/100
80/80 [==============================] - 250s 3s/step - loss: 1.0847e-05 - accuracy: 1.0000 - val_loss: 0.8087 - val_accuracy: 0.9092
Epoch 61/100
80/80 [==============================] - 203s 3s/step - loss: 1.0132e-05 - accuracy: 1.0000 - val_loss: 0.8135 - val_accuracy: 0.9092
Epoch 62/100
80/80 [==============================] - 195s 2s/step - loss: 9.6370e-06 - accuracy: 1.0000 - val_loss: 0.8183 - val_accuracy: 0.9084
Epoch 63/100
80/80 [==============================] - 197s 2s/step - loss: 9.2589e-06 - accuracy: 1.0000 - val_loss: 0.8238 - val_accuracy: 0.9092
Epoch 64/100
80/80 [==============================] - 194s 2s/step - loss: 8.5694e-06 - accuracy: 1.0000 - val_loss: 0

In [65]:
pred_y = conv_lstm_model.predict_classes(X_test_pad)
score=accuracy_score(y_test,pred_y)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 90.92%


In [64]:
confusion_matrix(y_test,pred_y)

array([[551,  60],
       [ 55, 601]])